In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum

class USBType(Enum):
    USB_A = "USB-A"
    USB_C = "USB-C"
    MICRO_USB = "Micro-USB"
    MINI_USB = "Mini-USB"
    UNKNOWN = "Unknown"

class USBClassifier:
    def __init__(self):
        # Parameters for feature extraction and classification
        self.debug_mode = True  # Set to True to display intermediate steps

    def preprocess_image(self, image_path):

        # Load image
        original = cv.imread(image_path)
        if original is None:
            raise Exception(f"Could not load image: {image_path}")

        # Keep a copy of the original for visualization
        result_img = original.copy()

        # Convert to grayscale
        gray = cv.cvtColor(original, cv.COLOR_BGR2GRAY)

        # Apply adaptive histogram equalization to improve contrast
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)

        # Apply Gaussian blur to reduce noise while preserving edges
        blurred = cv.GaussianBlur(enhanced, (5, 5), 0)

        # Apply additional median blur to handle salt-and-pepper noise if present
        median_blurred = cv.medianBlur(blurred, 5)

        # Try both Otsu's thresholding and adaptive thresholding
        _, binary_otsu = cv.threshold(median_blurred, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        binary_adaptive = cv.adaptiveThreshold(median_blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C,
                                             cv.THRESH_BINARY, 11, 2)

        # Combine the two methods (often gives better results)
        binary = cv.bitwise_and(binary_otsu, binary_adaptive)

        # Check if image might be inverted (more white than black)
        white_pixel_ratio = np.sum(binary == 255) / binary.size
        if white_pixel_ratio > 0.5:
            binary = cv.bitwise_not(binary)

        if self.debug_mode:
            self.display_steps("Preprocessing", [
                ("Original", original),
                ("Grayscale", gray),
                ("Enhanced", enhanced),
                ("Blurred", blurred),
                ("Binary (Otsu)", binary_otsu),
                ("Binary (Adaptive)", binary_adaptive),
                ("Binary (Combined)", binary)
            ])

        return original, gray, binary

    def extract_features(self, gray_img, binary_img):

        # Apply Canny edge detection with adjusted thresholds
        edges = cv.Canny(gray_img, 30, 150)

        # Morphological operations to clean up the binary image
        kernel = np.ones((5, 5), np.uint8)
        morphed = cv.morphologyEx(binary_img, cv.MORPH_CLOSE, kernel)
        morphed = cv.morphologyEx(morphed, cv.MORPH_OPEN, kernel)

        # Try different contour retrieval methods
        contours_ext, _ = cv.findContours(morphed, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        contours_all, _ = cv.findContours(morphed, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

        # Also try using edge image for contour detection
        contours_edges, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        # Combine all detected contours
        all_contours = contours_ext + contours_all + contours_edges

        if not all_contours:
            raise Exception("No contours found. Check the preprocessing steps.")

        # Filter contours by minimum area to avoid detecting noise
        min_area = 500  # Lower threshold to detect small USBs
        valid_contours = [cnt for cnt in all_contours if cv.contourArea(cnt) > min_area]

        if not valid_contours:
            raise Exception("No valid contours found after filtering. Check the preprocessing steps.")

        usb_contour = max(valid_contours, key=cv.contourArea)

        mask = np.zeros_like(gray_img)
        cv.drawContours(mask, [usb_contour], -1, 255, thickness=cv.FILLED)

        usb_region = cv.bitwise_and(gray_img, gray_img, mask=mask)

        rect = cv.minAreaRect(usb_contour)
        box = cv.boxPoints(rect)
        box = np.int32(box)

        width, height = rect[1]
        aspect_ratio = max(width, height) / min(width, height) if min(width, height) > 0 else 0

        area = cv.contourArea(usb_contour)
        perimeter = cv.arcLength(usb_contour, True)
        compactness = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0



        usb_edges = cv.Canny(usb_region, 20, 80)
        internal_contours, _ = cv.findContours(usb_edges, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)


        pin_candidates = []
        for cnt in internal_contours:
            # Adjust area thresholds for pin detection
            if cv.contourArea(cnt) > 5 and cv.contourArea(cnt) < area * 0.2:
                pin_candidates.append(cnt)
        horizontal_lines = 0
        if len(pin_candidates) > 0:
            for cnt in pin_candidates:
                x, y, w, h = cv.boundingRect(cnt)

                if w > 1.5*h and w > 8:
                    horizontal_lines += 1

        central_line_detected = False
        central_line_length = 0

        line_detection_img = usb_region.copy()

        central_lines_edges = cv.Canny(line_detection_img, 50, 150)

        lines = cv.HoughLinesP(central_lines_edges, 1, np.pi/180,
                              threshold=30, minLineLength=20, maxLineGap=10)

        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                line_length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                # Check if line is mostly horizontal
                if abs(y2 - y1) < 0.2 * abs(x2 - x1):
                    # Check if line is in the center region
                    center_y = mask.shape[0] // 2
                    if abs((y1 + y2) / 2 - center_y) < mask.shape[0] * 0.3:
                        central_line_detected = True
                        central_line_length = max(central_line_length, line_length)

        corner_img = gray_img.copy()
        corner_img = np.float32(corner_img)
        harris_corners = cv.cornerHarris(corner_img, blockSize=2, ksize=3, k=0.04)
        harris_corners = cv.dilate(harris_corners, None)

        corner_threshold = 0.01 * harris_corners.max()
        masked_corners = cv.bitwise_and(harris_corners, harris_corners, mask=mask)
        corner_count = np.sum(masked_corners > corner_threshold)

        hull = cv.convexHull(usb_contour, returnPoints=False)
        defects = None
        pin_count = 0

        try:
            if len(hull) > 3:  # Need at least 4 points for convexity defects
                defects = cv.convexityDefects(usb_contour, hull)
                if defects is not None:
                    # Filter significant defects (potential pins)
                    significant_defects = [defect for defect in defects if defect[0][3] > 1000]
                    pin_count = len(significant_defects)
        except:
            pass

        x, y, w, h = cv.boundingRect(usb_contour)
        bbox_area = w * h
        rectangularity = area / bbox_area if bbox_area > 0 else 0

        rows, cols = mask.shape

        center_col = cols // 2

        left_half = mask[:, :center_col]
        right_half = mask[:, center_col:]
        right_half_flipped = cv.flip(right_half, 1)

        min_width = min(left_half.shape[1], right_half_flipped.shape[1])
        left_half_cropped = left_half[:, -min_width:]
        right_half_flipped_cropped = right_half_flipped[:, :min_width]

        if left_half_cropped.size > 0 and right_half_flipped_cropped.size > 0:
            diff = cv.bitwise_xor(left_half_cropped, right_half_flipped_cropped)
            symmetry_score = np.sum(diff) / float(min_width * rows) if (min_width * rows) > 0 else 1.0
        else:
            symmetry_score = 1.0

        # Calculate roundedness at the ends (important for USB-C)
        # Use the contour to find the end curvature
        end_roundedness = 0.0
        try:
            # Fit ellipse to get roundedness information
            ellipse = cv.fitEllipse(usb_contour)
            major_axis, minor_axis = ellipse[1]
            # Roundedness is the ratio of minor to major (closer to 1 means more circular)
            if major_axis > 0:
                end_roundedness = minor_axis / major_axis
        except:
            pass

        # Create a feature dictionary
        features = {
            'contour': usb_contour,
            'box': box,
            'aspect_ratio': aspect_ratio,
            'area': area,
            'perimeter': perimeter,
            'compactness': compactness,
            'corner_count': corner_count,
            'pin_count': pin_count,
            'horizontal_lines': horizontal_lines,
            'symmetry_score': symmetry_score,
            'rectangularity': rectangularity,
            'mask': mask,
            'usb_region': usb_region,
            'usb_edges': usb_edges,
            'central_line_detected': central_line_detected,
            'central_line_length': central_line_length,
            'end_roundedness': end_roundedness
        }


        print(f"Extracted Features:")
        print(f"  - Aspect Ratio: {aspect_ratio:.2f}")
        print(f"  - Area: {area}")
        print(f"  - Perimeter: {perimeter:.2f}")
        print(f"  - Compactness: {compactness:.2f}")
        print(f"  - Corner Count: {corner_count}")
        print(f"  - Pin Count: {pin_count}")
        print(f"  - Horizontal Lines: {horizontal_lines}")
        print(f"  - Rectangularity: {rectangularity:.2f}")
        print(f"  - Symmetry Score: {symmetry_score:.8f}")
        print(f"  - Central Line Detected: {central_line_detected}")
        print(f"  - Central Line Length: {central_line_length:.2f}")
        print(f"  - End Roundedness: {end_roundedness:.2f}")

        if self.debug_mode:

            feature_vis = cv.cvtColor(gray_img, cv.COLOR_GRAY2BGR)
            cv.drawContours(feature_vis, [usb_contour], -1, (0, 255, 0), 2)
            cv.drawContours(feature_vis, [box], -1, (0, 0, 255), 2)


            internal_vis = cv.cvtColor(usb_region, cv.COLOR_GRAY2BGR)
            cv.drawContours(internal_vis, pin_candidates, -1, (0, 0, 255), 1)

            corner_vis = cv.cvtColor(gray_img, cv.COLOR_GRAY2BGR)
            corner_vis[harris_corners > corner_threshold] = [0, 0, 255]

            central_line_vis = cv.cvtColor(usb_region, cv.COLOR_GRAY2BGR)
            if lines is not None:
                for line in lines:
                    x1, y1, x2, y2 = line[0]
                    cv.line(central_line_vis, (x1, y1), (x2, y2), (0, 0, 255), 2)

            self.display_steps("Feature Extraction", [
                ("Edges", edges),
                ("Morphological Operations", morphed),
                ("Contour & Bounding Box", feature_vis),
                ("Internal Features", internal_vis),
                ("USB Edges", usb_edges),
                ("Harris Corners", corner_vis),
                ("USB Mask", mask),
                ("Central Lines", central_line_vis),
                ("Extracted USB Region", usb_region)
            ])

        return features
    def classify_usb_type(self, features):
        """
        Step 3: USB Type Classification
        - Analyze extracted features
        - Compare with predefined characteristics of each USB type
        - Make classification decision
        """
        aspect_ratio = features['aspect_ratio']
        compactness = features['compactness']
        corner_count = features['corner_count']
        pin_count = features['pin_count']
        symmetry_score = features['symmetry_score']
        area = features['area']
        horizontal_lines = features.get('horizontal_lines', 0)
        rectangularity = features.get('rectangularity', 0)
        central_line_detected = features.get('central_line_detected', False)
        central_line_length = features.get('central_line_length', 0)
        end_roundedness = features.get('end_roundedness', 0)

        # Print feature values for debugging
        print(f"DEBUG - Feature values for classification:")
        print(f"  Aspect Ratio: {aspect_ratio:.2f}")
        print(f"  Compactness: {compactness:.2f}")
        print(f"  Corner Count: {corner_count}")
        print(f"  Pin Count: {pin_count}")
        print(f"  Horizontal Lines: {horizontal_lines}")
        print(f"  Rectangularity: {rectangularity:.2f}")
        print(f"  Symmetry Score: {symmetry_score:.8f}")
        print(f"  Area: {area}")
        print(f"  Central Line Detected: {central_line_detected}")
        print(f"  Central Line Length: {central_line_length:.2f}")
        print(f"  End Roundedness: {end_roundedness:.2f}")


        scores = {
            USBType.USB_A: 0.0,
            USBType.USB_C: 0.0,
            USBType.MICRO_USB: 0.0,
            USBType.MINI_USB: 0.0
        }

        reasons = {
            USBType.USB_A: [],
            USBType.USB_C: [],
            USBType.MICRO_USB: [],
            USBType.MINI_USB: []
        }

        if 1.7 <= aspect_ratio <= 3.5:
            weight = 1.0 - min(abs(aspect_ratio - 2.5) / 1.0, 0.8)  # Ideal is around 2.5:1
            scores[USBType.USB_A] += weight * 0.30
            reasons[USBType.USB_A].append(f"Aspect ratio {aspect_ratio:.2f} fits USB-A range")


        if horizontal_lines >= 1:

            weight = min(horizontal_lines / 4.0, 1.0)
            scores[USBType.USB_A] += weight * 0.40
            reasons[USBType.USB_A].append(f"Detected {horizontal_lines} horizontal lines (pins)")

        if rectangularity > 0.65:
            weight = min((rectangularity - 0.65) / 0.25, 1.0)
            scores[USBType.USB_A] += weight * 0.20
            reasons[USBType.USB_A].append(f"Rectangularity {rectangularity:.2f} matches USB-A profile")

        if compactness > 0.6:
            weight = min((compactness - 0.6) / 0.3, 1.0)
            scores[USBType.USB_A] += weight * 0.10  # Reduced from 0.15 to 0.10
            reasons[USBType.USB_A].append(f"Compactness {compactness:.2f} fits USB-A")

        # USB-C scoring - IMPROVED
        if 2.8 <= aspect_ratio <= 4.0:
            weight = 1.0 - min(abs(aspect_ratio - 3.2) / 0.8, 0.8)  # Ideal is around 3.2:1
            scores[USBType.USB_C] += weight * 0.20  # Reduced from 0.25 to 0.20
            reasons[USBType.USB_C].append(f"Aspect ratio {aspect_ratio:.2f} fits USB-C range")

        # Symmetry is critical for USB-C
        if symmetry_score < 0.3:  # Lower score means more symmetric
            weight = 1.0 - min(symmetry_score / 0.3, 1.0)
            scores[USBType.USB_C] += weight * 0.25
            reasons[USBType.USB_C].append(f"High symmetry (score: {symmetry_score:.4f})")
        else:

            scores[USBType.USB_C] -= 0.15
            reasons[USBType.USB_C].append(f"Low symmetry (score: {symmetry_score:.4f})")

        if rectangularity > 0.75:
            weight = min((rectangularity - 0.75) / 0.2, 1.0)
            scores[USBType.USB_C] += weight * 0.15
            reasons[USBType.USB_C].append(f"High rectangularity {rectangularity:.2f}")

        if central_line_detected:
            relative_length = central_line_length / max(features['box'][0][0] - features['box'][2][0], 1)
            weight = min(relative_length, 1.0)
            scores[USBType.USB_C] += weight * 0.20  # Reduced from 0.25 to 0.20
            reasons[USBType.USB_C].append(f"Central line detected (strength: {weight:.2f})")

        if end_roundedness > 0.2:

            capped_roundedness = min(end_roundedness, 1.0)
            weight = min(capped_roundedness / 0.5, 1.0)
            scores[USBType.USB_C] += weight * 0.10
            reasons[USBType.USB_C].append(f"Rounded ends detected (score: {end_roundedness:.2f})")


        if horizontal_lines > 2:
            penalty = min(horizontal_lines / 5.0, 0.8)
            scores[USBType.USB_C] -= penalty * 0.25
            reasons[USBType.USB_C].append(f"Too many horizontal lines ({horizontal_lines}) for USB-C")

        if 3.8 <= aspect_ratio <= 5.2:
            weight = 1.0 - min(abs(aspect_ratio - 4.5) / 0.7, 0.8)  # Ideal is around 4.5:1
            scores[USBType.MICRO_USB] += weight * 0.4
            reasons[USBType.MICRO_USB].append(f"Aspect ratio {aspect_ratio:.2f} fits Micro-USB range")

        if compactness < 0.7:
            weight = min((0.7 - compactness) / 0.2, 1.0)
            scores[USBType.MICRO_USB] += weight * 0.3
            reasons[USBType.MICRO_USB].append(f"Lower compactness {compactness:.2f} typical of trapezoid shape")

        if corner_count > 6:
            weight = min(corner_count / 10.0, 1.0)
            scores[USBType.MICRO_USB] += weight * 0.2
            reasons[USBType.MICRO_USB].append(f"Corner count {corner_count} fits Micro-USB profile")

        if pin_count >= 2:
            weight = min(pin_count / 5.0, 1.0)
            scores[USBType.MICRO_USB] += weight * 0.1
            reasons[USBType.MICRO_USB].append(f"Detected {pin_count} pins")

        if 2.2 <= aspect_ratio <= 2.8:
            weight = 1.0 - min(abs(aspect_ratio - 2.5) / 0.3, 0.8)
            scores[USBType.MINI_USB] += weight * 0.4
            reasons[USBType.MINI_USB].append(f"Aspect ratio {aspect_ratio:.2f} fits Mini-USB range")

        if compactness < 0.75 and compactness > 0.5:
            weight = min((0.75 - compactness) / 0.25, 1.0)
            scores[USBType.MINI_USB] += weight * 0.3
            reasons[USBType.MINI_USB].append(f"Compactness {compactness:.2f} fits Mini-USB profile")

        if corner_count > 4 and corner_count < 8:
            weight = 1.0 - min(abs(corner_count - 6) / 2.0, 0.8)
            scores[USBType.MINI_USB] += weight * 0.3
            reasons[USBType.MINI_USB].append(f"Corner count {corner_count} fits Mini-USB profile")

        best_usb_type = max(scores.items(), key=lambda x: x[1])

        classification_threshold = 0.4

        if best_usb_type[1] >= classification_threshold:
            usb_type = best_usb_type[0]
            confidence = best_usb_type[1]
            reason = ", ".join(reasons[usb_type])
        else:
            usb_type = USBType.UNKNOWN
            confidence = 0.0
            reason = f"Could not classify USB type with sufficient confidence. Features: aspect ratio = {aspect_ratio:.2f}, " \
                    f"compactness = {compactness:.2f}, corner count = {corner_count}, " \
                    f"horizontal lines = {horizontal_lines}, rectangularity = {rectangularity:.2f}"

        print(f"Confidence scores:")
        for usb_type_key, score in scores.items():
            print(f"  {usb_type_key.value}: {score:.2f}")
            if score > 0:
                print(f"    Reasons: {', '.join(reasons[usb_type_key])}")

        return usb_type, confidence, reason

    def visualize_result(self, original_img, features, usb_type, confidence, reason):

        result_img = original_img.copy()

        cv.drawContours(result_img, [features['contour']], -1, (0, 255, 0), 2)
        cv.drawContours(result_img, [features['box']], -1, (0, 0, 255), 2)

        font = cv.FONT_HERSHEY_SIMPLEX
        text_position = (10, 30)
        cv.putText(result_img, f"Type: {usb_type.value}", text_position, font, 1, (0, 255, 0), 2)

        text_position = (10, 70)
        cv.putText(result_img, f"Confidence: {confidence:.2f}", text_position, font, 1, (0, 255, 0), 2)

        lines = reason.split(', ')
        y_pos = 110
        for line in lines:
            cv.putText(result_img, line, (10, y_pos), font, 0.6, (0, 200, 255), 2)
            y_pos += 30

        y_pos += 20
        cv.putText(result_img, f"Aspect Ratio: {features['aspect_ratio']:.2f}", (10, y_pos), font, 0.6, (255, 0, 0), 2)
        y_pos += 30
        cv.putText(result_img, f"Compactness: {features['compactness']:.2f}", (10, y_pos), font, 0.6, (255, 0, 0), 2)

        plt.figure(figsize=(12, 10))
        plt.imshow(cv.cvtColor(result_img, cv.COLOR_BGR2RGB))
        plt.title(f"USB Classification: {usb_type.value}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        return result_img

    def display_steps(self, stage_name, images):
        """
        Display intermediate steps for debugging and visualization
        """
        fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
        fig.suptitle(stage_name)

        for i, (title, img) in enumerate(images):
            if len(images) == 1:
                ax = axes
            else:
                ax = axes[i]

            if len(img.shape) == 2:  # Grayscale
                ax.imshow(img, cmap='gray')
            else:  # Color
                ax.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

            ax.set_title(title)
            ax.axis('off')

        plt.tight_layout()
        plt.show()

    def process_image(self, image_path):

        original, gray, binary = self.preprocess_image(image_path)

        features = self.extract_features(gray, binary)

        usb_type, confidence, reason = self.classify_usb_type(features)

        result_img = self.visualize_result(original, features, usb_type, confidence, reason)

        return usb_type, confidence, reason, result_img

if __name__ == "__main__":
    classifier = USBClassifier()
    image_path = '/content/minii.PNG'

    try:
        usb_type, confidence, reason, result_img = classifier.process_image(image_path)
        print(f"USB Type: {usb_type.value}")
        print(f"Confidence: {confidence:.2f}")
        print(f"Reason: {reason}")

        # Save result image if needed
        cv.imwrite('result.jpg', result_img)

    except Exception as e:
        print(f"Error processing image: {str(e)}")